## Retrieveing Data

In [2]:
import requests
import pandas as pd
import time
import json

CURRENT_TIME = round(time.time() * 1000) 

def jsonToDataFrame(json_resp):
    res = json.loads(json_resp)
    df = pd.DataFrame(res['result'])
    return df

def getSymbols():
    json_resp2 = (requests.get("https://fapi.binance.com/fapi/v1/exchangeInfo").json())
    for j in range(len(json_resp2['symbols'])):

        print(json_resp2['symbols'][j]['symbol'])

def getKlines(pair,interval,startime,endtime=CURRENT_TIME):
    query = { 
    "symbol":pair, 
    "interval": interval,
    "startTime": startime,
    "endTime": endtime,
    "limit":1000
    
    }
    # Add option for endtime
    
    json_resp = (requests.get("https://fapi.binance.com/fapi/v1/klines",params=query).json())
    df = pd.DataFrame(json_resp)
    return df


## Data extender

In [3]:
# Binance only gives you 1000 ticks at maximum , with this trick I extend that limit.
def dataExtender(pair,interval,initial_date,end_date): 
    list_of_dfs = []

    if interval == '15m':
        time_low = initial_date
        time_high = initial_date + (900000 *1000)

    elif interval == '1m':
        time_low = initial_date
        time_high = initial_date + (60000 *1000)

    if time_high > end_date:
        return getKlines(pair, interval, initial_date,end_date)

    while time_high < end_date:
        df = getKlines(pair, interval, time_low, time_high)
        list_of_dfs.append(df)
        time_low = time_high
        time_high += (60000 *1000)
    
    df = getKlines(pair, interval, time_low, end_date)
    list_of_dfs.append(df)
    result = pd.concat(list_of_dfs)
    
    return result
    

## Data Cleaning Aux

In [4]:
def dateToEpoch(date):
    a = date.split('/')
    timestamp = datetime(int('20' + a[2]),int(a[1]),int(a[0]),0,0).timestamp()
    return int(timestamp * 1000)

def epochToDate(epoch):
    # from datetime import datetime
    # print(datetime.fromtimestamp(int("1518308894652")/1000))
    return time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(epoch/1000))

def timestampToDay(epoch):
    day = time.strftime('%A', time.localtime(epoch))
    return day

def dayDf(date):
    # Use only with month and day. Ex: (04-20)
    temp_list = []
    for j in range(len(df) - 1):
        if date in df['open_time'].iloc[j]:
            temp_list.append(df.iloc[j])

    df = pd.DataFrame(temp_list)
    return df

def timeColumn(df):
    df_list = []
    for j in range(len(df)):
        # df['hour_of_day'].iloc[j] = int(df['open_time'].iloc[j][11:13])
        df_list.append(int(df['open_time'].iloc[j][11:13]))


    return df_list

def typeChanger(df):

    df['open_timestamp'] = df['open_timestamp'].astype('int')
    df['open'] = df['open'].astype('float')
    df['high'] = df['high'].astype('float')
    df['low'] = df['low'].astype('float')
    df['close'] = df['close'].astype('float')
    df['volume'] = df['volume'].astype('float')
    df['number_of_trades'] = df['number_of_trades'].astype('float')

    return df

def vol_amplitude_cat(df):
    vol_amplitude = df['volume'].max() - df['volume'].min()

    bins = [df['volume'].min(), vol_amplitude/4 , vol_amplitude/4*2, vol_amplitude/4*3, df['volume'].max()]
    names = ['Muy Bajo VOL', 'Medio VOL', 'Alto VOL', 'Muy Alto VOL', ]

    df['vol_cat'] = pd.cut(df['volume'], bins, labels=names)

def candleColorColumn(df):
    df['candle_color'] = [0 if df['close'].iloc[j] < df['open'].iloc[j] else 1 for j in range(len(df))]
    df['candle_color_debug'] = ["RED" if df['close'].iloc[j] < df['open'].iloc[j] else "GREEN" for j in range(len(df))]

def open_month_and_year(df):
    result_list = []
    for j in range(len(df)):
        print(df['open_time'].iloc[j][0:4])
        result_list.append(int(df['open_time'].iloc[j][0:4]))

    
    result = pd.concat(result_list,df)

    return result

## Data Cleaning Main

In [5]:
def dataCleaning(df):
    columns = {
        0:'open_time',
        6:'close_time',
        1:'open',
        2:'high',
        3:'low',
        4:'close',
        5:'volume',
        8:'number_of_trades'
    }

    df = df.drop(columns=[7,9,10,11])
    df = df.rename(columns = columns)
    
    # df = df.sort_values(by=['time'])
    df['open_timestamp'] = df['open_time']
    df['open_time'] = df['open_time'].apply(epochToDate)
    df['close_time'] = df['close_time'].apply(epochToDate)
    df['day_of_week'] = df['open_timestamp'].apply(timestampToDay) 
    

    # df = open_month_and_year(df)

    first_column = df.pop('close_time')
    # insert column using insert(position,column_name,
    # first_column) function
    df.insert(1, 'close_time', first_column)

    df.insert(2,'hour_of_day', timeColumn(df) )
    # this might not be an optimal approach for large datasets, but for this size it suffices

    # Transforming columns into correct type
    # timeColumn(df)
    typeChanger(df)
    candleColorColumn(df)
    # Sorting
    df = df.sort_values(by=['open_timestamp'])

    return df

## Candlestick graph

In [6]:
import plotly.graph_objects as go

import pandas as pd
from datetime import datetime

def candleGraph(df):
    fig = go.Figure(data=[go.Candlestick(x=df['time'],
                    open=df['open'],
                    high=df['high'],
                    low=df['low'],
                    close=df['close'])])

    fig.show()


## Data exploration

In [7]:
def dataPipeline(pair,interval,initial_date,end_date):
    print("Retrieving data...")
    df = dataExtender(pair,interval,initial_date, end_date)
    print('Data retrieved succesfully...')
    print('Starting cleaning...')
    df = dataCleaning(df)
    df.reset_index(drop=True ,inplace=True)
    print('Ended cleaning...')
    return df

In [ ]:
lista_dfs_15m =[]
for j in range(1,12):
    print('15m')
    initial_date = dateToEpoch("28/{}/21".format(j))
    end_date = dateToEpoch("28/{}/21".format(j+1))
    print(epochToDate(initial_date),epochToDate(end_date))
    lista_dfs_15m.append(("Mes {}".format(j),dataPipeline('LUNAUSDT','15m',initial_date,end_date)))

In [ ]:
lista_dfs_1m =[]
for j in range(1,12):
    print('1m')
    initial_date = dateToEpoch("30/{}/21".format(j))
    end_date = dateToEpoch("30/{}/21".format(j+1))
    print(epochToDate(initial_date),epochToDate(end_date))
    lista_dfs_1m.append(dataPipeline('LUNAUSDT','1m',initial_date,end_date))

1m
2021-11-01 00:00:00 2021-12-30 00:00:00
Retrieving data...
Data retrieved succesfully...
Starting cleaning...
Ended cleaning...


## Whole year

In [ ]:
initial_date = dateToEpoch("1/1/21")
end_date = dateToEpoch("30/12/22")

result_15m = dataPipeline("LUNAUSDT",'15m',initial_date,end_date)
result_1m = dataPipeline("LUNAUSDT",'1m',initial_date,end_date)

Retrieving data...
Data retrieved succesfully...
Starting cleaning...
Ended cleaning...
Retrieving data...
Data retrieved succesfully...
Starting cleaning...
Ended cleaning...


## Exploration

In [12]:
def rrrgTick(df):
    df['rrrg'] = [
        1 if 
        df['candle_color'].iloc[j-3] == 0 and 
        df['candle_color'].iloc[j-2] == 0 and 
        df['candle_color'].iloc[j-1] == 0 and 
        df['candle_color'].iloc[j] == 1
        else 0 if
        df['candle_color'].iloc[j-3] == 0 and 
        df['candle_color'].iloc[j-2] == 0 and 
        df['candle_color'].iloc[j-1] == 0 and 
        df['candle_color'].iloc[j] == 0
        else 12
        for j in range(len(df))
        ]

def grrrgTick(df):
    df['grrrg'] = [
        1 if 
        df['candle_color'].iloc[j-4] == 1 and 
        df['candle_color'].iloc[j-3] == 0 and 
        df['candle_color'].iloc[j-2] == 0 and 
        df['candle_color'].iloc[j-1] == 0 and 
        df['candle_color'].iloc[j] == 1
        else 0 if
        df['candle_color'].iloc[j-4] == 1 and 
        df['candle_color'].iloc[j-3] == 0 and 
        df['candle_color'].iloc[j-2] == 0 and 
        df['candle_color'].iloc[j-1] == 0 and 
        df['candle_color'].iloc[j] == 0
        else 12
        for j in range(len(df))
        ]

def rrrTick(df):
    df['rrr'] = [
        1 if 
        df['candle_color'].iloc[j-2] == 0 and 
        df['candle_color'].iloc[j-1] == 0 and 
        df['candle_color'].iloc[j] == 0
        else 0
        for j in range(len(df))
        ]

In [37]:
def strat1Earnings(df,pattern,initial_balance,verbose=False):

    rrrTick(df)

    winable_amount = 1
    winable_percent = 0
    balance = initial_balance
    fee = 0.03
    winners = 0
    loosers = 0
    operation_list = []

    for j in range(len(df)):
        
        if df[pattern].iloc[j-1] == 1 and df[pattern].iloc[j-2] != 1:
            #print(df['open_time'].iloc[j-1])
            # print(winable_percent)
            # print((df['close'].iloc[j] - df['open'].iloc[j]) * 100 / (df['open'].iloc[j])
            round_percent_delta = (df['close'].iloc[j] - df['open'].iloc[j]) * 100 / (df['open'].iloc[j])
            balance += (balance * round_percent_delta / 100) - fee
            
            if round_percent_delta > 0:
                winners += 1
                operation_list.append(('WINNER --- Compra: {} en fecha: {}'.format(df['close'].iloc[j-2],df['open_time'].iloc[j-2]), 'Venta: {} en fecha: {}'.format(df['close'].iloc[j],df['open_time'].iloc[j])))
                # print(df['open_time'].iloc[j-2] , 'WINNER')

            else:
                # print(df['open_time'].iloc[j-2] , 'LOOSER')
                loosers += 1
                operation_list.append(('LOOSER --- Compra: {}'.format(df['close'].iloc[j-2]), 'Venta: {}'.format(df['close'].iloc[j])))
                
    percentage = balance * 100 / initial_balance - 100
        
    winrate = winners / (winners + loosers) * 100
    
    result = [balance - initial_balance, percentage]
    
    if verbose == True:
        print("Pattern: " + pattern)
        print("DF has: {} elements".format(len(df)))
        print("Winners: {} --- Loosers: {} --- Winrate: {}".format(winners,loosers, winrate))
        print("Initial Balance: {} --- End Balance: {} -- Percentage: {} ".format(initial_balance, balance, percentage))     

    return result

def strat2Earnings(df,pattern,initial_balance,verbose=False):
    
    rrrTick(df)
    
    winable_amount = 1
    winable_percent = 0
    balance = initial_balance
    winners = 0
    loosers = 0
    fee = 0.03

    operation_list = []

    for j in range(len(df)):
        
        if df[pattern].iloc[j-2] == 1 and df[pattern].iloc[j-3] != 1:
            # print(winable_percent)
            # print((df['close'].iloc[j] - df['open'].iloc[j]) * 100 / (df['open'].iloc[j])
            round_percent_delta = (df['close'].iloc[j] - df['close'].iloc[j-2]) * 100 / (df['open'].iloc[j])
            balance += (balance * round_percent_delta / 100 ) - fee

            if round_percent_delta > 0:
                winners += 1
                operation_list.append(('WINNER --- Compra: {} en fecha: {}'.format(df['close'].iloc[j-2],df['open_time'].iloc[j-2]), 'Venta: {} en fecha: {}'.format(df['close'].iloc[j],df['open_time'].iloc[j])))
                # print(df['open_time'].iloc[j-2] , 'WINNER')

            else:
                # print(df['open_time'].iloc[j-2] , 'LOOSER')
                loosers += 1
                operation_list.append(('LOOSER --- Compra: {}'.format(df['close'].iloc[j-2]), 'Venta: {}'.format(df['close'].iloc[j])))

    percentage = round(balance * 100 / initial_balance - 100)
    
    winrate = winners / (winners + loosers) * 100
    
    result = [balance - initial_balance, percentage]

    if verbose == True:  
        print("Pattern: " + pattern)
        print("DF has: {} elements".format(len(df)))  
        print("Winners: {} --- Loosers: {} --- Winrate: {}".format(winners,loosers, winrate))
        print("Initial Balance: {} --- End Balance: {} -- Percentage: {} ".format(initial_balance, balance, percentage))     

    return result, operation_list

In [38]:
def stratsSimulator(initial_date,end_date):
    # initial_date = dateToEpoch("1/1/22")
    # end_date = dateToEpoch("1/3/22")

    initial_date = dateToEpoch(initial_date)
    end_date = dateToEpoch(end_date)

    result_15m = dataPipeline("LUNAUSDT",'15m',initial_date,end_date)
    result_1m = dataPipeline("LUNAUSDT",'1m',initial_date,end_date)

    data_list_1, op_list_1 = strat1Earnings(result_15m,'rrr',1000)
    data_list_2, op_list_2 = strat1Earnings(result_1m,'rrr',1000)
    data_list_3, op_list_3 = strat2Earnings(result_15m,'rrr',1000)
    data_list_4, op_list_4 = strat2Earnings(result_1m,'rrr',1000)

    print("Strat 1: 15m ", data_list_1)
    print("Strat 1: 1m ", data_list_2)
    print("Strat 2: 15m ", data_list_3)
    print("Strat 2: 1m ", data_list_4)


In [43]:
stratsSimulator("1/3/22","9/3/22")

Retrieving data...
Data retrieved succesfully...
Starting cleaning...
Ended cleaning...
Retrieving data...
Data retrieved succesfully...
Starting cleaning...
Ended cleaning...
Strat 1: 15m  [29.29647711561074, 2.929647711561074]
Strat 1: 1m  [-101.15230770937751, -10.115230770937742]
Strat 2: 15m  [18.594542559074284, 2]
Strat 2: 1m  [-60.31735836732355, -6]


In [41]:
stratsSimulator("1/2/22","1/3/22")

Retrieving data...
Data retrieved succesfully...
Starting cleaning...
Ended cleaning...
Retrieving data...
Data retrieved succesfully...
Starting cleaning...
Ended cleaning...
Strat 1: 15m  [160.05429161882603, 16.005429161882617]
Strat 1: 1m  [21.786078773726103, 2.178607877372599]
Strat 2: 15m  [272.66021436104324, 27]
Strat 2: 1m  [438.30899993791013, 44]


In [59]:
stratsSimulator("2/2/22","1/3/22")

Retrieving data...
Data retrieved succesfully...
Starting cleaning...
Ended cleaning...
Retrieving data...
Data retrieved succesfully...
Starting cleaning...
Ended cleaning...
Strat 1: 15m  [72.65252405683646, 7.26525240568364]
Strat 1: 1m  [6.898157173234495, 0.6898157173234551]
Strat 2: 15m  [187.52060145656287, 19]
Strat 2: 1m  [284.110538573618, 28]


In [48]:
stratsSimulator('1/12/21','1/3/22')

Retrieving data...
Data retrieved succesfully...
Starting cleaning...
Ended cleaning...
Retrieving data...
Data retrieved succesfully...
Starting cleaning...
Ended cleaning...
Strat 1: 15m  [601.2703466153964, 60.12703466153965]
Strat 1: 1m  [-335.10947405639695, -33.5109474056397]
Strat 2: 15m  [292.81025380902247, 29]
Strat 2: 1m  [1057.3209529770893, 106]


In [ ]:
for j in range(len(lista_dfs_15m)):
    data_list = strat1Earnings_alt(lista_dfs_15m[j],'rrr',100)
    print(data_list)

1.155608549006189
21.26518919663762
-3.16546618422565
-14.447363034379805
7.910908366936383
13.304165287098911
2.3309898920368823
25.38971788599619
-11.903569982291629
6.301124286069012
17.0460472239683


In [ ]:
def tickSize(df):
    df['tick_size'] = df['high'] - df['low']
    df['tick_size_percentage'] = df['tick_size'] * 100 / df['low']

,open_time,close_time,hour_of_day,open,high,low,close,volume,number_of_trades,open_timestamp,day_of_week,candle_color,candle_color_debug,rrrg,grrrg,rrr
0,2021-11-01 00:00:00,2021-11-01 00:14:59,0,42.109,42.327,42.024,42.212,73932.0,4431.0,1635735600000,Friday,1,GREEN,12,12,0
1,2021-11-01 00:15:00,2021-11-01 00:29:59,0,42.207,42.386,42.169,42.362,51770.0,3781.0,1635736500000,Tuesday,1,GREEN,12,12,0
2,2021-11-01 00:30:00,2021-11-01 00:44:59,0,42.361,42.561,42.311,42.410,41217.0,3576.0,1635737400000,Friday,1,GREEN,12,12,0
3,2021-11-01 00:45:00,2021-11-01 00:59:59,0,42.410,42.433,41.774,41.869,122516.0,6488.0,1635738300000,Tuesday,0,RED,12,12,0
4,2021-11-01 01:00:00,2021-11-01 01:14:59,1,41.885,42.006,41.556,41.967,113103.0,7480.0,1635739200000,Friday,1,GREEN,12,12,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2885,2021-11-30 23:00:00,2021-11-30 23:14:59,23,56.849,57.198,56.647,56.802,155412.0,8875.0,1638324000000,Thursday,0,RED,0,0,1
2886,2021-11-30 23:15:00,2021-11-30 23:29:59,23,56.805,56.831,56.508,56.801,155440.0,9310.0,1638324900000,Sunday,0,RED,0,12,1
2887,2021-11-30 23:30:00,2021-11-30 23:44:59,23,56.807,57.290,56.752,57.289,146706.0,8317.0,1638325800000,Thursday,1,GREEN,1,12,0
2888,2021-11-30 23:45:00,2021-11-30 23:59:59,23,57.289,57.572,56.952,57.068,129699.0,8874.0,1638326700000,Sunday,0,RED,12,12,0


In [ ]:
result_15m = pd.concat(lista_dfs_15m[-2:13])
result_1m = pd.concat(lista_dfs_1m[-2:13])

In [ ]:
result_15m

,open_time,close_time,hour_of_day,open,high,low,close,volume,number_of_trades,open_timestamp,day_of_week,candle_color,candle_color_debug,rrrg,grrrg,rrr
0,2021-10-01 00:00:00,2021-10-01 00:14:59,0,37.325,37.670,37.259,37.400,148852.0,6482.0,1633057200000,Monday,1,GREEN,12,12,0
1,2021-10-01 00:15:00,2021-10-01 00:29:59,0,37.396,37.645,37.379,37.593,83917.0,4318.0,1633058100000,Friday,1,GREEN,12,12,0
2,2021-10-01 00:30:00,2021-10-01 00:44:59,0,37.601,37.702,37.492,37.596,101836.0,4249.0,1633059000000,Monday,0,RED,12,12,0
3,2021-10-01 00:45:00,2021-10-01 00:59:59,0,37.609,37.698,37.348,37.643,110605.0,5352.0,1633059900000,Friday,1,GREEN,12,12,0
4,2021-10-01 01:00:00,2021-10-01 01:14:59,1,37.657,37.877,37.620,37.817,91253.0,5789.0,1633060800000,Monday,1,GREEN,12,12,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2885,2021-11-30 23:00:00,2021-11-30 23:14:59,23,56.849,57.198,56.647,56.802,155412.0,8875.0,1638324000000,Thursday,0,RED,0,0,1
2886,2021-11-30 23:15:00,2021-11-30 23:29:59,23,56.805,56.831,56.508,56.801,155440.0,9310.0,1638324900000,Sunday,0,RED,0,12,1
2887,2021-11-30 23:30:00,2021-11-30 23:44:59,23,56.807,57.290,56.752,57.289,146706.0,8317.0,1638325800000,Thursday,1,GREEN,1,12,0
2888,2021-11-30 23:45:00,2021-11-30 23:59:59,23,57.289,57.572,56.952,57.068,129699.0,8874.0,1638326700000,Sunday,0,RED,12,12,0


In [ ]:
data_list_1 = strat1Earnings_alt(result_15m,'rrr',1000)
data_list_2 = strat1Earnings_alt(result_1m,'rrr',1000)
data_list_3 = strat2Earnings_alt(result_15m,'rrr',1000)
data_list_4 = strat2Earnings_alt(result_1m,'rrr',1000)

print(data_list_1)
print(data_list_2)
print(data_list_3)
print(data_list_4)

1947.124536297013
-1272.5728979897171
1814.7539453684462
-78754.94613062969
